In [1]:
import numpy as np
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
df = pd.read_csv("/home/root495/Inexture/CDSS-RAG/data/raw/conversation_summary.csv")
df.head()



,conversation,summary
0,"Doctor: Hello? Hi. Um, should we start? Yeah, ...","3/7 hx of diarrhea, mainly watery. No blood in..."
1,Doctor: Hello? Patient: Hello. Can you hear me...,"4/7 hx of dry itchy skin, mainly on chest and ..."
2,Doctor: Hello? Patient: Hello. Doctor: Hello t...,"Headache on left side. Started few hours ago, ..."
3,"Doctor: Alex. Ohh. Hello? Hi, can you hear me?...","4/7 hx of generally unwell, mainly sore throat..."
4,Doctor: Hello? Patient: Doctor: . Good morning...,2/7 ago developed lower abdo pain/suprapubic p...


In [78]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
You are a medical summarization expert. Given the following conversation between a doctor and a patient, and additional relevant clinical context (if available), generate a concise, accurate, and structured medical note summarizing the history, symptoms, relevant negatives, medical/social/family history, impression, and plan.
If clinical context retrieved via RAG is available, use it to make your summary more complete and accurate. If no additional context is provided, base your summary solely on the conversation.

Your summary should match the concise, professional style of a real clinical note and include important details while omitting irrelevant conversation.

Format the summary in a single paragraph, like this example:
'3/7 hx of diarrhea, mainly watery. No blood in stool. Opening bowels x6/day. Associated LLQ pain - crampy, intermittent, nil radiation. Also vomiting - mainly bilous. No blood in vomit. Fever on first day, nil since. Has been feeling lethargic and weak since. Takeaway 4/7 ago - Chinese restaurant. Wife and children also unwell with vomiting, but no diarrhea. No other unwell contacts. PMH: Asthma DH: Inhalers SH: works as an accountant. Lives with wife and children. Affecting his ADLs as has to be near toilet often. Nil smoking/etOH hx Imp: gastroenteritis Plan: Conservative management - rest, push fluids, paracetamol if feverish. Recommend OTC diarolyte. To review in 3-5d if symptoms not improving. To see earlier if feeling more unwell.'

Your answer will be evaluated using cosine similarity, BLEU, and ROUGE scores—so accuracy and inclusion of key clinical details matter.

If no relevant context is retrieved, answer based only on the conversation. If context is provided, integrate it to improve your summary.

Context:
{context}

Conversation:
{question}
""")

In [85]:
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.2,
    max_tokens=2048,
    api_key=groq_api_key
)

response = llm.invoke("Explain RAG in simple words.")
print(response)


content="RAG (Retrieval, Augment, Generate) is a simple yet powerful concept in artificial intelligence, especially in language models. Here's a simplified explanation:\n\nImagine you're trying to answer a question or write a text. A RAG system works in three steps:\n\n1. **Retrieval**: The system searches for relevant information from a large database or knowledge base. It's like looking up a book or article that might have the answer.\n2. **Augment**: The system takes the retrieved information and adds more context or details to it. This helps to make the information more complete and accurate.\n3. **Generate**: Finally, the system uses the augmented information to generate a response or text. This could be an answer to a question, a summary of a topic, or even a creative piece of writing.\n\nThink of RAG like a research assistant. It helps you find the right information, adds more context to it, and then uses that information to create a useful response. This process makes it easier

In [80]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# Initialize embedding model as in preprocess.ipynb
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Load Chroma DB as retriever
chroma_db = Chroma(
    persist_directory="/home/root495/Inexture/CDSS-RAG/notebooks/chroma_store",
    embedding_function=embedding_model
)
retriever = chroma_db.as_retriever()


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [86]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)


In [82]:
response = rag_chain.invoke("Doctor: Hello? Hi. Um, should we start? Yeah, okay. Hello how um. Good morning sir, how can I help you this morning? Patient: Hello, how are you? Patient: Oh hey, um, I've just had some diarrhea for the last three days, um, and it's been affecting me I need to stay close to the toilet. And, um, yeah, it's been affecting my day-to-day activities. Doctor: Sorry to hear that. Um, and and when you say diarrhea, what'd you mean by diarrhea? Do you mean you're going to the toilet more often? Or are your stools more loose? Patient: Yeah, so it's like loose and watery stool, going to the toilet quite often, uh and like some pain in my, like, lower stomach? Doctor: Doctor: Okay. And how many times a day are you going, let's say, in the last couple of days? Patient: Um, probably like six or seven times a day? Yeah. Doctor: Six, seven times a day. And you mention it's mainly watery. Have you noticed any other things, like blood in your stools? Patient: No, no blood, yeah, just watery and loose stool. Doctor: Okay. And you mentioned you've had some pain in your tummy as well. Whereabouts is the pain, exactly? Patient: Yep. Patient: So in my lower abdomen, so, uh, like, um...yeah, just to one side. Doctor: One side. And what side is that? Patient: Uh, on the left side. Doctor: Left side. Okay, and can you describe the pain to me? Patient: Yeah, it feels, um, like a cramp, like a muscular cramp, and, um, yeah i feel a bit uh weak and shaky. Doctor: Okay. And is the pain, is that, is it there all the time, or does it come and go? Patient: Uh, it comes and goes. Doctor: Come and go. Does the pain move anywhere else, for example towards your back? Patient: Uh...no, just maybe my stomach. Doctor: Okay, fine. And you mentioned you've been feeling quite weak and shaky as well. What do you mean by shaky? Do you mean you've been having, uh have you been feeling feverish, for example? Patient: Yeah. Patient: Um, yeah, it doesn't feel like -- yeah, it just makes me feel weak. I haven't had a fever, um, at the moment, but I did notice um a temperature when the symptoms started, so, um, yeah around about three or four days ago. Doctor: Doctor: You measure your temperature then? Patient: Yeah, I uh I didn't mention my temperature, no, but I felt, um, just a bit hot. And, y'know. Doctor: Okay. Okay. Any other symptoms like sweating, or um, night sweats? No? And, uh, any vomiting at all? Patient: Uh, no. Patient: Yeah, so um, I vomited at the start of the symptoms but now um I've stopped vomiting. Doctor: You stopped vomiting, okay. And was your vomit, I know it's not a nice thing to talk about, but was it just normal food colour Yeah. And there was no blood in your vomit, is that right? Patient: Yeah, yeah, just normal vomit, yeah. No no blood, no. Yeah. Doctor: No, okay. Um, and um, any any other symptoms at all? So you mentioned tummy pain, you mentioned diarrhea, you mentioned your vomiting, uh, anything else that comes to mind? Patient: Yep. Um, I had a loss of appetite, um, so I haven't been eating as much, but I've been able to hold down fluids. Doctor: Okay. Doctor: Okay, so you're drinking fluids. Um, what kind of foods have you managed to eat, if anything? Patient: Yep. Patient: Um, just soups, and, uh, yeah, light foods. Like smoothies and, yeah, liquid foods mainly. Doctor: Okay. Fine. Um, and sir these started three days ago the symptoms. Are you aware of any triggers which may have caused the symptoms uh to kick on. So for example, think like takeaway foods or eating out or being around other people with similar symptoms. Patient: Patient: Yeah, so I had takeaway about four days ago, um, uh, but other than that I've, yeah, been, uh, eating normally. Nothing unusual here. Doctor: Okay. Doctor: Do you remember where you ate? Patient: Um, yeah, I ate at a Chinese restaurant with friends. Yeah. Doctor: Okay. Anyone else unwell with similar symptoms? Patient: Um, so no one else in the family, so a wife and two kids and one, um, child was vomiting, but they haven't got diarrhea. There's no one with the same symptoms. Doctor: Okay, okay. Fine. Um, alright. And uh, in terms of your , your overall health, are you normally fit and well? Or, uh Patient: Um, yeah, I mean, other than um athsma, um I use an inhaler, everything uh else is fine. Doctor: Okay. And, is your asthma well-controlled? Patient: Uh, yeah, that's fine. I just, yeah, use an inhaler, and uh that's under control. Doctor: Fine. And you don't have any other tummy problem, bowel problems I should be aware of? Patient: No. Doctor: No, okay. Um, and apart from the inhalers, do you take any other medications? Patient: Uh, no, no other medications. Doctor: Okay, fine. And in terms of just your day to day life, you said it's been affecting your life, um, in what way has it been affecting your life? Patient: Yeah. Patient: Uh, so, I need to stay close to the toilet 'cause I go quite frequently during the these past three days. Um, yeah, other than that, it's uh, yeah, the main concern. Doctor: Okay. Doctor: Yeah. Doctor: And have you, are you currently working at the moment? Patient: Uh, yes, yeah. I I work, er. Um, I'm an accountant. Doctor: Would, would work. Doctor: Okay. Have you been going into work the last three days, or have you been at home? Patient: Uh, yeah, I've been going to work. Yeah. Yeah, it's been quite difficult. Doctor: okay. That must be difficult for you then. Doctor: fine. And you said, you mentioned you live with your wife and two children, is that right? Patient: Yes, yeah. Doctor: Right, alright. Um, just a couple of other question we need to ask, sir. Um, do you smoke at all? Patient: Uh, no, I don't smoke. Doctor: And do you drink much in the way of alcohol? Patient: Uh, no, I I don't drink alcohol, no. Doctor: Okay. so um, er normally at this stage I like to um, examine you if that's okay, but um, um, but but having listened to your story, sir, I think uh, um, just to recap for the last three days you've been having loose stool, diarrhea, a bit of tummy pain uh mainly on the left-hand side, um and vomiting and fever and you're quite weak and lethargic um, you mentioned you had this Chinese takeaway as little as three days ago and I wondered whether that might be the cause of your problems. Patient: Yeah. Patient: Patient: Okay. Doctor: Um, it seems like you may have something, uh, called gastroenteritis, which essentially just a tummy bug or infection of your uh of your tummy. Patient: Doctor: Uh, mainly caused by viruses but there can be a possibility of bacteria uh causing its symptoms. Um. Patient: Yeah. Patient: Yeah. Doctor: At this stage, uh, what, what we'd recommend is just what we say conservative management. So, um, I don't think you need anything like antibiotics. It's really just, um, making sure you're well hydrated, so drinking fluids. Patient: Patient: Mm-hmm. Doctor: Um, there are things like Dioralyte you can get from the pharmacy, which uh it's um it helps helps replenish your minerals and vitamins. Patient: Okay. Doctor: Um, and if you are having vomiting diarrhea I would say recommend that in the first, you know, first couple of days. Patient: Yep. Doctor: If you are feeling feverish and weak, eh taking some paracetamol, uh, two tablets up to four times a day for the first few days can also help. Patient: Yep. Doctor: I will certainly advise you to take some time off work, actually I know you're quite keen to work but I would say the next two, two to three days as the infection clears from your system to take some time off and rest. Patient: Okay. Patient: Yeah. Doctor: Um, I'll admit if your symptoms haven't got better, you know, in in three to four days, I'd like to come and see you again. Patient: Okay, sure. Doctor: Because if it is ongoing then we have to wonder whether something else caused your symptoms. Patient: Yep. Doctor: Uh, and we may need to do further tests like um taking a sample of your stool so we can test that. Patient: Doctor: Um, etcetera etcetera. Patient: Yep, sure, yep. Doctor: How's that sound? Patient: That sounds great, yeah. Yeah. Doctor: Do you have any questions for me? Patient: Um, no, no further questions, no. Doctor: Okay, and is uh is the treatment plan clear? Patient: Uh, yes, yeah, that's that's very clear. Thank you. Doctor: Great. Well, I wish you all the best. Patient: Okay, thank you. Bye. Doctor: Thank you. Bye bye.")
print(response)


content='3-day history of diarrhea, mainly watery, with 6-7 episodes per day, associated with crampy, intermittent left lower quadrant pain, and initial vomiting, now resolved. No blood in stool or vomit. Patient reports feeling weak and lethargic, with an initial fever, now resolved. Recent history of takeaway food consumption, specifically Chinese restaurant, 4 days prior to symptom onset, with one child experiencing vomiting, but no other family members with similar symptoms. PMH: Asthma, well-controlled with inhalers. SH: Non-smoker, non-drinker, works as an accountant, and lives with wife and two children. The symptoms have been affecting his daily activities, requiring him to stay close to the toilet. Imp: Gastroenteritis, likely viral. Plan: Conservative management with rest, hydration, and consideration of Dioralyte for mineral and vitamin replenishment. Paracetamol recommended if feverish, up to 4 times a day. Patient advised to take 2-3 days off work to recover. Follow-up in 

In [83]:
response.content

'3-day history of diarrhea, mainly watery, with 6-7 episodes per day, associated with crampy, intermittent left lower quadrant pain, and initial vomiting, now resolved. No blood in stool or vomit. Patient reports feeling weak and lethargic, with an initial fever, now resolved. Recent history of takeaway food consumption, specifically Chinese restaurant, 4 days prior to symptom onset, with one child experiencing vomiting, but no other family members with similar symptoms. PMH: Asthma, well-controlled with inhalers. SH: Non-smoker, non-drinker, works as an accountant, and lives with wife and two children. The symptoms have been affecting his daily activities, requiring him to stay close to the toilet. Imp: Gastroenteritis, likely viral. Plan: Conservative management with rest, hydration, and consideration of Dioralyte for mineral and vitamin replenishment. Paracetamol recommended if feverish, up to 4 times a day. Patient advised to take 2-3 days off work to recover. Follow-up in 3-4 days

In [87]:
rag_summary = []
num = 0
for i in df["conversation"]:
    if num >= 10:
        break
    rag_summary.append(rag_chain.invoke(i).content)
    print(i)
    num += 1
    print(num)


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jq1h0y5ke5va8r76fmwt8rer` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 94809, Requested 5383. Please try again in 2m45.888s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [ ]:

df["rag_summary"] = rag_summary
df.to_csv("/home/root495/Inexture/CDSS-RAG/data/processed/conversation_summary.csv", index=False)

ValueError: Length of values (2) does not match length of index (57)

In [68]:
from transformers import AutoTokenizer, AutoModel
import torch

# Use a sentence-transformer or similar embedding model from HuggingFace
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

text = "3-day history of watery diarrhea, occurring 6-7 times a day, with associated left lower quadrant abdominal pain described as crampy and intermittent, but no radiation. Initial episode of vomiting, now resolved, with no blood in stool or vomit. Patient reports feeling weak and lethargic, with an initial fever that has since resolved. Recent history of eating at a Chinese restaurant 4 days prior to symptom onset, with one child experiencing vomiting, but no other family members with similar symptoms. Patient has a history of asthma, well-controlled with inhalers, and works as an accountant, currently struggling to manage symptoms while at work. He lives with his wife and two children, does not smoke, and does not consume alcohol. Impression is gastroenteritis, likely viral in nature. Plan includes conservative management with rest, hydration, and consideration of Dioralyte for mineral and vitamin replenishment. Paracetamol is recommended if patient becomes feverish, and a follow-up visit is scheduled in 3-5 days to reassess symptoms, with potential for further testing if symptoms persist."

# Tokenize and get embeddings
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
with torch.no_grad():
    outputs = model(**inputs)
    # Mean pooling
    generated = outputs.last_hidden_state.mean(dim=1).squeeze()



In [75]:
text = '3/7 hx of diarrhea, mainly watery. No blood in stool. Opening bowels x6/day. Associated LLQ pain - crampy, intermittent, nil radiation. Also vomiting - mainly bilous. No blood in vomit. Fever on first day, nil since. Has been feeling lethargic and weak since. Takeaway 4/7 ago - Chinese restaurant. Wife and children also unwell with vomiting, but no diarrhea. No other unwell contacts. PMH: Asthma DH: Inhalers SH: works as an accountant. Lives with wife and children. Affecting his ADLs as has to be near toilet often. Nil smoking/etOH hx Imp: gastroenteritis Plan: Conservative management - rest, push fluids, paracetamol if feverish. Recommend OTC diarolyte. To review in 3-5d if symptoms not improving. To see earlier if feeling more unwell.'
# Tokenize and get embeddings
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
with torch.no_grad():
    outputs = model(**inputs)
    # Mean pooling
    actual = outputs.last_hidden_state.mean(dim=1).squeeze()

In [76]:
print(text)

3/7 hx of diarrhea, mainly watery. No blood in stool. Opening bowels x6/day. Associated LLQ pain - crampy, intermittent, nil radiation. Also vomiting - mainly bilous. No blood in vomit. Fever on first day, nil since. Has been feeling lethargic and weak since. Takeaway 4/7 ago - Chinese restaurant. Wife and children also unwell with vomiting, but no diarrhea. No other unwell contacts. PMH: Asthma DH: Inhalers SH: works as an accountant. Lives with wife and children. Affecting his ADLs as has to be near toilet often. Nil smoking/etOH hx Imp: gastroenteritis Plan: Conservative management - rest, push fluids, paracetamol if feverish. Recommend OTC diarolyte. To review in 3-5d if symptoms not improving. To see earlier if feeling more unwell.


In [73]:
import torch

# Compute cosine similarity between 'actual' and 'generated'
cosine_sim = torch.nn.functional.cosine_similarity(actual, generated, dim=0)
print("Cosine similarity between 'actual' and 'generated':", cosine_sim.item())


Cosine similarity between 'actual' and 'generated': 0.6797723770141602
